In [1]:
import os
import sys
sys.path.append(os.path.abspath('../../../src/'))

import pandas as pd
import numpy as np

import SortedNoDupeBedOverlap as bdo
import io_utils


In [2]:
for seed in range(1000):
    np.random.seed(seed)
    chrom_size = 1000
    
    ## simulate random intervals
    ints0 = np.sort(np.random.choice(chrom_size + 1, size = 100, replace = False))
    ints1 = np.sort(np.random.choice(chrom_size + 1, size = 100, replace = False))
    
    bed0 = [('chr1', a, b) for a, b in zip(ints0[::2], ints0[1::2])]
    bed1 = [('chr1', a, b) for a, b in zip(ints1[::2], ints1[1::2])]
    
    ## compute pairwise overlap in a brute force way
    bed0_overlaps_bf = []
    for i, (c0, a0, b0) in enumerate(bed0):
        overlap = sum([bdo.overlap(a0, b0, a1, b1)  if c0 == c1 else 0 for (c1, a1, b1) in bed1])
        bed0_overlaps_bf.append(overlap / (b0 - a0))
    
    ## compute pairwise overlap with a linear algorithm
    overlaps_linear = bdo.BedOverlap(bed0, bed1, 0.5)   

    assert set([p for p, o in zip(bed0, bed0_overlaps_bf) if o >= 0.5]) == set(overlaps_linear)